# Amazon OpenSearch hands-on lab
본 워크샵을 통해 Amazon DynamoDB를 SageMaker Notebook 또는 SageMaker Studio 환경에서 간단히 실습합니다.

AWS Workshop Studio의 ['Dive into OpenSearch Service - 모듈 1'](https://catalog.us-east-1.prod.workshops.aws/workshops/f0213896-4dd9-494a-89c5-f7886b45ed4a/ko-KR/search)을 참고하였습니다. 

## 0. Setup Environment

In [ ]:
!pip install boto3 -U

In [ ]:
# OpenSearch 사용자 이름 및 비밀번호를 지정 합니다.
username = 'admin'
password = 'Passw0rd!'

## 1. OpenSearch 도메인 생성

In [ ]:
import boto3
import time

def create_opensearch_domain(domain_name):
    client = boto3.client('opensearch')

    response = client.create_domain(
        DomainName=domain_name,
        EngineVersion='OpenSearch_2.17',
        ClusterConfig={
            'InstanceType': 'r6g.large.search',
            'InstanceCount': 2,
            'DedicatedMasterEnabled': False,
            'ZoneAwarenessEnabled': False,
        },
        EBSOptions={
            'EBSEnabled': True,
            'VolumeType': 'gp3',
            'VolumeSize': 100,
        },
        DomainEndpointOptions={
            'EnforceHTTPS': True,
            'TLSSecurityPolicy': 'Policy-Min-TLS-1-2-2019-07',
        },
        NodeToNodeEncryptionOptions={
            'Enabled': True,
        },
        EncryptionAtRestOptions={
            'Enabled': True,
        },
        AdvancedSecurityOptions={
            'Enabled': True,
            'InternalUserDatabaseEnabled': True,
            'MasterUserOptions': {
                'MasterUserName': username,
                'MasterUserPassword': password
            }
        },
    )

    print(response)

    endpoint_url = wait_for_domain_creation(client, domain_name)

    return endpoint_url


def wait_for_domain_creation(client, domain_name, wait_period=60, retries=30):
    """
    OpenSearch 도메인이 활성화될 때까지 기다린 후 도메인 엔드포인트 URL 반환

    :param client: boto3 OpenSearch client
    :param domain_name: OpenSearch 도메인 이름
    :param wait_period: 각 시도 사이에 대기할 시간 (초)
    :param retries: 최대 시도 횟수
    :return: 도메인 엔드포인트 URL
    """
    for _ in range(retries):
        describe_response = client.describe_domain(DomainName=domain_name)
        if 'Endpoint' in describe_response['DomainStatus']:
            return describe_response['DomainStatus']['Endpoint']
        time.sleep(wait_period)
    raise Exception("Domain creation timeout")

In [ ]:
# OpenSearch 도메인 생성 (약 15분 소요)

domain_name = 'my-opensearch-domain1'
endpoint_url = create_opensearch_domain(domain_name)

## 2. 도메인 보안 설정 업데이트 (AWS Console에서 작업)

<b>생성한 도메인 보안 설정을 수정합니다.</b><br>
새로운 탭에서 [Amazon OpenSearch 콘솔](https://us-east-1.console.aws.amazon.com/aos/home?#opensearch/domains)을 엽니다.<br>

생성된 도메인의 이름을 선택 합니다.<br>
![](./img/os-edit-security-config1.png)

화면 우측 상단의 <b>Actions</b>에서 Edit security configuration을 선택 합니다.<br>
![](./img/os-edit-security-config2.png)

<b>Access Policy</b>에서 Domain access policy 부분에서 <b>Only use fine-grained access control</b>을 선택한 뒤, 페이지 맨 하단의 <b>Save Changes</b>를 클릭 합니다.
![](./img/os-edit-security-config3.png)

## 3. Index 설정

먼저 데이터를 가져올 인덱스를 만듭니다.

### 3.1 Index 생성

In [ ]:
endpoint_url

In [ ]:
# OpenSearch Service의 엔드포인트 URL
endpoint = 'https://'+endpoint_url

먼저 데이터를 가져올 인덱스를 만듭니다.
<br>

```
PUT /my-movie-index
{
  "settings": {
    "index": {
      "number_of_shards": 3,
      "number_of_replicas": 1
    }
  }
}

```

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import boto3


# PUT 요청 URL
url = endpoint + '/my-movie-index'

# HTTP 헤더
headers = { "Content-Type": "application/json" }

# 요청 본문
payload = {
  "settings": {
    "index": {
      "number_of_shards": 3,
      "number_of_replicas": 1
    }
  }
}

response = requests.put(url, auth=HTTPBasicAuth(username, password), json=payload, headers=headers)

# 응답 확인
print(response.status_code)
print(response.text)


### 3.2 Index 생성 확인

```
GET /my-movie-index 

```

In [ ]:
# GET 요청 URL
url = endpoint + '/my-movie-index'

# HTTP 헤더
headers = { "Content-Type": "application/json" }

response = requests.get(url, auth=HTTPBasicAuth(username, password), headers=headers)

# 응답 확인
print(response.status_code)
print(response.text)


## 4. 데이터 삽입

로컬에 위치한 ./data/imdb-data-for-indexing.txt 내용을 OpenSearch에 넣습니다.

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json

# POST 요청 URL (my-movie-index의 movies 타입에 문서 삽입)
url = endpoint + '/my-movie-index/_doc/_bulk'

# HTTP 헤더
headers = { "Content-Type": "application/x-ndjson" }

# 삽입할 데이터 파일 경로
data_file_path = './data/imdb-data-for-indexing.txt'

# 파일을 열고 각 줄을 JSON 형태로 읽기
with open(data_file_path, 'r') as file:
    lines = file.readlines()
    bulk_data = ''

    for i in range(1, len(lines), 2):  # 1에서 시작하여 2씩 증가
        index_line = json.loads(lines[i])
        data_line = lines[i+1]

        bulk_data += json.dumps(index_line) + '\n' + data_line  # \n으로 index line과 data line 구분

    response = requests.post(url, auth=HTTPBasicAuth(username, password), headers=headers, data=bulk_data.encode('utf-8'))

    # 응답 확인
    print(response.status_code)
    print(response.text)


## 5. 데이터 쿼리하기

### 5.1 심플 쿼리

매개변수 없이 검색 API를 실행하면 필터링 없이 인덱스에서 결과를 제공합니다.<br>
검색 필터가 제공하지 않은 모든 문서가 검색 결과에 포함됩니다.<br><br>

```
GET /my-movie-index/_search
```
<br>

방금 검색한 데이터는 다음 두 단계에서 수행됩니다.

- 쿼리 단계 - 이 단계에서는 쿼리가 각 샤드에서 로컬로 실행되고 일치하는 문서 ID가 쿼리 노드로 반환됩니다. 쿼리 노드는 이러한 레코드를 병합하고 정렬된 목록을 만듭니다.
- 패칭 단계 - 이 단계에서 쿼리 노드는 ID에서 실제 데이터를 가져와 클라이언트에 반환합니다.



In [ ]:
import requests
from requests.auth import HTTPBasicAuth

# GET 요청 URL
url = endpoint + '/my-movie-index/_search'

response = requests.get(url, auth=HTTPBasicAuth(username, password))

# 응답 확인
print(response.status_code)
print(response.text)


[참고]
OpenSearch에서 쿼리를 실행할 때 여러분은 엔드포인트로 요청을 보내고, 이는 라운드 로빈 형식으로 OpenSearch 클러스터의 데이터 노드에 요청됩니다. 한 노드가 요청을 받으면, 이를 해당 쿼리에 대한 코디네이터 노드라고 하며 요청된 인덱스에 대한 샤드를 저장하는 실제 데이터 노드에서 데이터 검색을 시작합니다. 코디네이터 노드는 쿼리되는 인덱스와 관련된 데이터를 갖고 있을 수도 있고 갖고있지 않을 수도 있습니다.

OpenSearch는 간단한 URL 기반 쿼리 구문과 더 복잡한 REST 기반 JSON 인터페이스를 지원합니다. <br>
OpenSearch를 사용할 때 JSON API를 가장 자주 사용하게 됩니다. 살짝 물에 발을 담궈보고 싶다면, 다음 URL 기반 쿼리를 실행하여 쿼리 매개변수로 간단한 검색을 실행합니다.<br><br>

```
GET /my-movie-index/_search?q=franco
```
<br>

OpenSearch에서 'franco'가 포함된 모든 문서를 검색하는 GET 요청을 실행합니다. 이 요청은 'franco'라는 단어가 포함된 문서를 검색하므로, 검색 결과에는 'franco'라는 단어가 언급된 모든 문서가 포함됩니다. 

In [ ]:
import requests
from requests.auth import HTTPBasicAuth

# GET 요청 URL
url = endpoint + '/my-movie-index/_search?q=franco'

response = requests.get(url, auth=HTTPBasicAuth(username, password))

# 응답 확인
print(response.status_code)
print(response.text)


### 5.2 Term 쿼리

term 쿼리는 검색어와 정확히 일치하는 문서를 반환합니다. <br>
출력을 검사하고 반환된 모든 문서에 <i>directors.keyword</i> 용어가 포함되어 있는지 확인합니다. 키워드 값을 다양하게 변형해 보십시오.<br><br>

```
GET my-movie-index/_search
{
  "query": {
    "term": {
      "directors.keyword": {
        "value": "James Franco"
      }
    }
  }
}
```
<br>


In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json

# GET 요청 URL
url = endpoint + '/my-movie-index/_search'

# 검색 쿼리
query = {
    "query": {
        "term": {
            "directors.keyword": {
                "value": "James Franco"
            }
        }
    }
}

response = requests.get(url, auth=HTTPBasicAuth(username, password), json=query)

# 응답 확인
print(response.status_code)
print(json.dumps(response.json(), indent=4))


### 5.3 Keyword를 이용한 쿼리

OpenSearch에는 다양한 쿼리 유형을 지원하는 풍부한 쿼리 인터페이스가 있습니다.<br>
- Term 쿼리 - 용어를 정확히 일치시키려는 경우 이 쿼리 계열을 사용합니다. keyword 필드와 함께 용어 쿼리를 자주 사용합니다.
- Match 쿼리 - 긴 텍스트에서 매칭하고 특히 OpenSearch의 관련성을 사용하여 결과를 정렬할 때 이 쿼리 계열을 사용합니다. text 필드와 함께 이러한 쿼리를 가장 자주 사용합니다.
<br>
그렇다면 <span style='color: #2D3748; background-color: #fff5b1'>keyword</span> 필드와 <span style='color: #2D3748; background-color: #fff5b1'>text</span> 필드의 차이점은 무엇입니까? <br>
OpenSearch는 analysis 라고 하는 프로세스로 text 필드의 문자열을 처리합니다. 분석하는 동안 OpenSearch는 텍스트를 <i>세그먼트(segment)화</i> 하여 개별 <i>용어(term)</i> 를 추출합니다. 용어를 공백으로 구분된 단어로 생각할 수 있지만 일부 언어, 예를 들어 일본어, 한국어와 같은 아시아 언어에는 더 복잡하고 상황 인지가 필요한 세분화된 내용이 있습니다. OpenSearch는 용어의 스트림을 추가로 처리하여 구성 가능한 변환 세트를 적용하고, <i>불용어(stop word)</i> 라는 일반적인 단어를 제거하고, 단어를 공통 어간 형식으로 줄이고, 동의어를 추가하고, 소문자로 변환하는 등의 작업을 수행합니다.
<br><br>
Keyword 필드는 매칭을 위한 단일 용어를 제공합니다. OpenSearch는 <span style='color: #2D3748; background-color: #fff5b1'>Keyword</span> 필드에 텍스트 변환을 적용하지 않습니다.
<br>

```
GET my-movie-index/_search
{
  "query": {
    "term": {
      "title": "transformers"
    }
  }
}
```

<br>

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json

# GET 요청 URL
url = endpoint + '/my-movie-index/_search'

# 검색 쿼리
query = {
    "query": {
        "term": {
            "title": "transformers"
        }
    }
}

response = requests.get(url, auth=HTTPBasicAuth(username, password), json=query)

# 응답 확인
print(response.status_code)
print(json.dumps(response.json(), indent=4))

>[Note]<br>
>전달된 텍스트는 분석기를 통해 소문자로 변환됩니다. Transformers에 대해 위의 쿼리에서 transformers로 검색한 것을 확인하십시오. 즉, term 쿼리(토큰화된 단어에 대한 대소문자 구분 검색)를 수행할 때 transformers로 검색하면 일치된 항목을 찾지만 Transformers로 검색은 찾을 수 없습니다.


<span style='color: #2D3748; background-color: #fff5b1'>_analyze</span> API를 사용하여 OpenSearch가 입력한 텍스트를 어떻게 변환하는지 확인할 수 있습니다.<br>
```
GET my-movie-index/_analyze
{
  "analyzer": "default",
  "text": ["Transformers"]
}
```

이 쿼리는 "Transformers"라는 텍스트에 대해 OpenSearch의 "default" 분석기를 사용하여 텍스트 분석을 실행합니다. 

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json

# POST 요청 URL (GET 대신 POST를 사용해야 분석을 실행할 수 있습니다)
url = endpoint + '/my-movie-index/_analyze'

# 분석 요청 본문
body = {
    "analyzer": "default",
    "text": ["Transformers"]
}

response = requests.post(url, auth=HTTPBasicAuth(username, password), json=body)

# 응답 확인
print(response.status_code)
print(json.dumps(response.json(), indent=4))


>[Note]<br>
>대소문자를 구분하지 않는 검색의 경우 [simple_query_string](https://opensearch.org/docs/latest/query-dsl/full-text/) 작업을 사용할 수 있습니다. simple_query_string은 검색어의 match 계열에 있습니다.


```
POST my-movie-index/_search
{
  "query": {
    "simple_query_string": {
      "query": "Transformers",
      "fields": ["title"]
    }
  }
}
```
<br>
이 쿼리는 "Transformers"라는 텍스트를 "title" 필드에서 검색하는 것입니다.

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json

# POST 요청 URL (my-movie-index 검색)
url = endpoint + '/my-movie-index/_search'


# 검색 요청 본문
body = {
    "query": {
        "simple_query_string": {
            "query": "Transformers",
            "fields": ["title"]
        }
    }
}

response = requests.post(url, auth=HTTPBasicAuth(username, password), json=body)

# 응답 확인
print(response.status_code)
print(json.dumps(response.json(), indent=4))


### 5.4 Range 쿼리

Range 쿼리는 숫자 값에 대해 동작하며 쿼리에서 지정한 범위에 속하는 일치 항목을 반환합니다. 다음 비교 연산자를 사용할 수 있습니다.<br>
- 보다 큼: gt
- 보다 작음: lt
- 크거나 같음: gte
- 작거나 같음: lte
<br><br>
다음 쿼리는 2014와 2016 사이에 출시된 영화를 반환합니다. 다른 조건으로도 변형을 시도해 보십시오.
<br>
```
GET my-movie-index/_search
{
  "query": {
    "range": {
      "year": {
        "gt": 2014,
        "lt": 2016
      }
    }
  }
}
```

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json

# POST 요청 URL (my-movie-index 검색)
url = endpoint + '/my-movie-index/_search'

# 검색 요청 본문
body = {
    "query": {
        "range": {
            "year": {
                "gt": 2014,
                "lt": 2016
            }
        }
    }
}

response = requests.get(url, auth=HTTPBasicAuth(username, password), json=body)

# 응답 확인
print(response.status_code)
print(json.dumps(response.json(), indent=4))


<br><br>이전과 마찬가지로 range 쿼리를 다른 유형의 쿼리와 결합할 수 있습니다.<br>
이 쿼리는 "year" 필드의 값이 2014 초과이면서 동시에 2016 미만이며, "genres" 필드에 "action"이 포함된 문서를 검색합니다. 
```
GET my-movie-index/_search
{
  "query": {
    "bool": {
      "must": [
        {
          "range": {
            "year": {
              "gt": 2014,
              "lt": 2016
            }
          }
        },
        {
          "match": {
            "genres": "action"
          }
        }
      ]
    }
  }
}

```


In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json

# POST 요청 URL (my-movie-index 검색)
url = endpoint + '/my-movie-index/_search'


# 검색 요청 본문
body = {
    "query": {
        "bool": {
            "must": [
                {
                    "range": {
                        "year": {
                            "gt": 2014,
                            "lt": 2016
                        }
                    }
                },
                {
                    "match": {
                        "genres": "action"
                    }
                }
            ]
        }
    }
}

response = requests.get(url, auth=HTTPBasicAuth(username, password), json=body)

# 응답 확인
print(response.status_code)
print(json.dumps(response.json(), indent=4))


### 5.5 Boost 쿼리

OpenSearch는 점수 알고리즘을 사용하여 문서를 반환하기 전에 점수를 매기고 정렬합니다. boosting 을 사용하여 점수에 영향을 줄 수 있습니다. 필드를 부스트하면 해당 필드에서 일치하는 용어의 점수에 부스트 계수를 곱합니다.<br><br>

다음 예는 부스팅 전후의 샘플 쿼리를 보여줍니다. 부스팅 없이 출력 결과 세트는 검색어 "horror"에 대한 문서의 기본 순서를 갖게 됩니다.<br>
이 쿼리는 "title"과 "plot" 필드 모두에서 "horror"를 검색합니다. 
<br>
```
GET my-movie-index/_search
{
  "query": {
    "multi_match": {
      "query": "horror",
      "fields": ["title","plot"]
    }
  }
}

```

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json

# POST 요청 URL (my-movie-index 검색)
url = endpoint + '/my-movie-index/_search'

# 검색 요청 본문
body = {
    "query": {
        "multi_match": {
            "query": "horror",
            "fields": ["title","plot"]
        }
    }
}

response = requests.get(url, auth=HTTPBasicAuth(username, password), json=body)

# 응답 확인
print(response.status_code)
print(json.dumps(response.json(), indent=4))


<br>
부스트 기능을 사용하면 캐럿(^) 연산자를 사용하여 쿼리에 부스팅 요소를 추가하여 영화 제목 필드에서 찾은 결과에 더 큰 가중치를 할당할 수 있습니다.<br>
예를 들어 "title^2"는 이 필드에서 일치 항목의 중요성을 두 배로 늘립니다. "title^3" 는 3배가 됩니다. 부스팅 적용 전과 후 결과의 _score를 비교합니다.<br>

```
GET my-movie-index/_search
{
  "query": {
    "multi_match": {
      "query": "horror",
      "fields": ["title^2","plot"]
    }
  }
}
```


In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json

# POST 요청 URL (my-movie-index 검색)
url = endpoint + '/my-movie-index/_search'

# 검색 요청 본문
body = {
    "query": {
        "multi_match": {
            "query": "horror",
            "fields": ["title^2","plot"]
        }
    }
}

response = requests.get(url, auth=HTTPBasicAuth(username, password), json=body)

# 응답 확인
print(response.status_code)
print(json.dumps(response.json(), indent=4))


<br>
title 필드에서 일치 항목의 가중치를 높임으로써 이제 결과의 순서가 변경되었음을 확인할 수 있습니다.<br>

>참고<br>
>검색에 OpenSearch를 사용할 때, 일반적으로 중요도에 따라 필드에 가중치를 부여합니다. title, product_name 등과 같은 필드는 일반적으로 description, plot, comments 등과 같은 설명 필드보다 일치에 더 중요합니다.

## 6. 리소스 정리

이 모듈에서는 추가 요금이 발생하지 않도록 이 단원에서 만든 리소스를 정리합니다.<br>
아래 코드를 실행시켜 생성되었던 OpenSearch Domain을 삭제 합니다.

다만, 이렇게 도메인을 삭제하면 데이터 및 설정이 모두 제거되므로, 운영환경에서의 삭제는 신중하게 고려해야 합니다.<br>
도메인이 삭제될 때까지 수 분 시간이 걸릴 수 있습니다.

In [ ]:
def delete_opensearch_domain(domain_name):
    client = boto3.client('opensearch')

    response = client.delete_domain(
        DomainName=domain_name
    )

    print(response)

# OpenSearch 도메인 삭제
delete_opensearch_domain(domain_name)


아래 셀이 완료되면 OpenSearch Domain은 삭제가 완료됩니다.

In [ ]:
# 삭제 상태 확인
client = boto3.client('opensearch')
while True:
    try:
        response = client.describe_domain(DomainName=domain_name)
        print(f"Domain status: {response['DomainStatus']['Processing']}")

        if response['DomainStatus']['Deleted'] == True:
            print(f"Domain {domain_name} Status : Deleting")

    except client.exceptions.ResourceNotFoundException:
        print(f"Domain {domain_name} not found. It may have been successfully deleted.")
        break

    except Exception as e:
        print(f"Unexpected error: {e}")
        break

    time.sleep(30)  # 30초마다 상태 확인
